# Metodo Simplex

Usamos Python para realizar el método Simplex para minimizar y maximizar para la forma canónica
Para ello nececsitaremos metodos en Arrays

In [7]:
# OPERACION FILA EN UNA MATRIZ  

def OperacionFila(Matriz,f1,f2,k=1):
    n = Matriz[0].__len__()
    for i in range(n):
        Matriz[f1][i] += k*Matriz[f2][i]

In [9]:
# OPERACION FILA POR CONSTANTE
def FilaPorConstante(Matriz,f,k):
    for i in range(Matriz[0].__len__()):
        Matriz[f][i] *= k

In [4]:
# HALLAR EL INDICE DE UN ELEMENTO EN UN ARRAY DADO
def IndiceEnArray(n,Array):
    indice = 0
    for i in range(Array.__len__()):
        if n == Array[i]:
            return indice
        indice+=1
    return indice

In [5]:
# RETORNA UN ARRAY CON LOS INDICES EN LOS QUE APARECE CIERTO VALOR EN EL ARRAY
def IndicesConValorEnArray(Array,n):
    indices = []
    for i in range(Array.__len__()):
        if n==Array[i]:
            indices.append(i)
    return indices

In [6]:
# NUMERO DE VECES QUE APARECE CIERTO VALOR EN UN ARRAY
def VecesApareceEnArray(Array,n):
    Nveces = 0
    for i in range(Array.__len__()):
        if(n==Array[i]):
            Nveces +=1
    return Nveces

In [16]:
# VERIFICAR SI HAY CEROS EN UN ARRAY
def ArrayConCeros(Array):
    for i in range(Array.__len__()):
        if Array[i]==0:
            return True
    return False

## Obtener la Tabla 

Para ello se crean dos funciones, uno para crear la tabla cuando se desee maximizar y otra cuando se desea minimizar

In [12]:
# TABLA PARA MAXIMIZAR
def ConstruirTablaMaximizar(MatrizA,MatrizB,MatrizZ):
    n = MatrizA.__len__()
    m = 1
    p = MatrizZ.__len__()
    Tabla = np.zeros((n+1,1+m+p+n))
    for i in range(n):
        Tabla[i][0] = 0
        ind = 1
        for j in range(p):
            Tabla[i][ind] = MatrizA[i][j]
            ind+=1
        for j in range(n):
            if i==j:
                Tabla[i][ind]=1
            else:
                Tabla[i][ind]=0
            ind+=1
        Tabla[i][ind] = MatrizB[i][0]
    Tabla[n][0] = 1
    for i in range(1,1+m+p+n):
        if i <= MatrizZ.__len__():
            Tabla[n][i] = -1 * MatrizZ[i-1]
        else:
            Tabla[n][i] = 0
        
    return Tabla


In [14]:
def ConstruirTablaMinimizar(MatrizA,MatrizB,MatrizZ):
    n = MatrizA.__len__() # Numero de Ecuaciones de Restriccion
    m = 1 # Una columna de B
    p = MatrizZ.__len__() # Numero de Variables
    Tabla = np.zeros((n+1,p+m+2*n))
    for i in range(n):
        ind = 0
        for j in range(p):
            Tabla[i][ind] = MatrizA[i][j]
            ind+=1
        for j in range(n*2):
            if j==2*i:
                Tabla[i][ind] = -1
            elif j==2*i+1:
                Tabla[i][ind] = 1
            else:
                Tabla[i][ind] = 0
            ind+=1
        Tabla[i][ind] = MatrizB[i][0]
    for i in range(p+m+2*n):
        if i<p:
            Tabla[n][i] = MatrizZ[i]
        else:
            if (i-p)%2==0 and i-p>=0:
                Tabla[n][i] = 0
            else:
                Tabla[n][i] = 1
    return Tabla

Para encontrar el pivote en ambas tablas se necesitan ciertas funciones, para hallar la columna y la fila en la que se encuentra

### Para la tabla de Maximizar

In [17]:
# MAYOR ABSOLUTO EN UN ARRAY (ULTIMA FILA DE LA TABLA)
def MayorAbsoluto(Array):
    n = Array.__len__()
    Fila = np.zeros(n-1)
    for i in range(n-1):
        Fila[i] = Array[i]
    FilaAbs = list(map(lambda x:abs(x),Fila))
    maxVal = max(FilaAbs)
    # Encontrar el indice 
    ind = 0
    for i in range(Fila.__len__()):
        if abs(Fila[i]) == maxVal:
            ind = i
            break
    # Retorna el maximo valor y el indice en el que se encuentra
    return maxVal,ind

### Para la tabla de minimizar

In [18]:
def EncontrarMenorEnFila(Matriz,ColMultiply,NumeroVariables):
    indiceColumna = 0
    n = Matriz[0].__len__()
    ArraySumasDeConstante = np.zeros(n-1)
    ArraySumasNoConstante = np.zeros(n-1)
    for i in range(n-1):
        sumaConst = 0
        sumaNoConst = 0
        for j in range(ColMultiply.__len__()):
            if ColMultiply[j] == 0:
                sumaConst += Matriz[j][i]
            else:
                sumaNoConst += ColMultiply[j]*Matriz[j][i]
            
        ArraySumasDeConstante[i] = -1 *sumaConst
        ArraySumasNoConstante[i] = -1 * sumaNoConst
        if (i - NumeroVariables) % 2 == 1 and i-NumeroVariables>=0:
            ArraySumasDeConstante[i] += 1
        else:
            ArraySumasNoConstante[i] += Matriz[Matriz.__len__()-1][i]
    MinVal = min(ArraySumasDeConstante)
    indiceColumna = IndiceEnArray(MinVal,ArraySumasDeConstante)
    if VecesApareceEnArray(ArraySumasDeConstante,MinVal)>1:
        indices = IndicesConValorEnArray(ArraySumasDeConstante,MinVal)
        MinVal = ArraySumasNoConstante[indices[0]]
        for ind in indices:
            if ArraySumasNoConstante[ind]<MinVal:
                MinVal = ArraySumasNoConstante[ind]
        indiceColumna = IndiceEnArray(MinVal,ArraySumasNoConstante)
    condicion = False
    if FilaNoNegativa(ArraySumasDeConstante) and FilaNoNegativa(ArraySumasDeConstante):
        condicion = True
    return indiceColumna,condicion

En ambos necesito encontrar el pivote ya se tiene el indice de la columna, el proceso para ambos es igual por lo que se usa la misma función

In [20]:
def EncontrarPivote(Tabla,indiceColumna):
    n = Tabla.__len__()
    m = Tabla[0].__len__()
    ColumnaDivision = np.zeros(n-1)
    for i in range(n-1):
        if Tabla[i][indiceColumna]==0:
            ColumnaDivision[i] = -1
        else:
            ColumnaDivision[i] = Tabla[i][m-1] / Tabla[i][indiceColumna]
    minimaDivision = max(ColumnaDivision)
    for i in range(n-1):
        if ColumnaDivision[i]<0:
            continue
        if ColumnaDivision[i]<minimaDivision:
            minimaDivision = ColumnaDivision[i]
    indiceFila = 0
    for i in range(n-1):
        if ColumnaDivision[i] == minimaDivision:
            indiceFila = i
    pivote = Tabla[indiceFila][indiceColumna]
    return pivote,indiceFila


Para el caso de Maximizar se tiene una condicion que detiene el bucle el cual es que no haya elementos negativos. En el caso de minimizar se obtiene esta condición en la función que encuentra la columna del pivote

In [21]:
def FilaNoNegativa(Array):
    condicion = True
    for i in range(Array.__len__()):
        if Array[i]<0:
            condicion = False
            return condicion
    return condicion

Luego ese pivote debe ser 1, por lo que se usa la operacion fila por una constante, y la columna de dicho pivote debe ser cero

In [23]:
# PARA MAXIMIZAR
def ColumnasDelPivoteCero(Matriz,indFila,indCol):
    n = Matriz.__len__()
    for i in range(n):
        if i == indFila:
            continue
        k = Matriz[i][indCol] / Matriz[indFila][indCol]
        OperacionFila(Matriz,i,indFila,-k)
# PARA MINIMIZAR
def ColumnaDelPivoteCero(Matriz,indFila,indCol):
    n = Matriz.__len__()
    for i in range(n-1):
        if i == indFila:
            continue
        k = Matriz[i][indCol] / Matriz[indFila][indCol]

Se tiene dos funciones similares sin embargo en el metodo para minimizar no se afecta la ultima fila

Y estos metodos los usamos en los metodos simplex

### METODO SIMPLEX MAXIMIZAR

In [27]:
# Metodo Simplex para maximizar
def SimplexMaximizar(Tabla,Solucion):
    n = Tabla.__len__()
    m = Tabla[0].__len__()
    while not FilaNoNegativa(Tabla[n-1]):
        indCol = MayorAbsoluto(Tabla[n-1])[1]
        pivote, indFila = EncontrarPivote(Tabla,indCol)
        FilaPorConstante(Tabla,indFila,1/pivote) 
        ColumnasDelPivoteCero(Tabla,indFila,indCol)
        Solucion[indFila][0] = indCol 
    for i in range(Solucion.__len__()):
        if Solucion[i][0]!=0:
            Solucion[i][1] = Tabla[i][m-1]
    return Tabla[n-1][m-1]


Probamos el valor optimo con la siguiente función

In [28]:
def ProbarOptimoValor(Solucion,z,SolMaxima):
    VariablesColumna = np.zeros((z.__len__()))
    for i in range(z.__len__()):
        VariablesColumna[i] = Solucion[i][0]
    ValMax = 0
    for i in range(Solucion.__len__()):
        ind = int(Solucion[i][0])-1
        ValMax += Solucion[i][1] * z[ind] 
        if VariablesColumna[i] != 0:
            indice = int(Solucion[i][0])
            print(f"La variable x{indice} es igual a",round(Solucion[i][1],1))
    if ArrayConCeros(VariablesColumna):
        print("Las otras variables son iguales a 0")
    if ValMax == SolMaxima:
        print("La solucion maxima es : " ,round(SolMaxima,1))
    else:
        print("No se encontra la solucion maxima")

### METODO SIMPLEX MINIMIZAR

In [29]:
# Metodo Simplex para minimizar
def SimplexMinimizar(Tabla,Solucion,NumeroVariables):
    n = Tabla.__len__()
    m = Tabla[0].__len__()
    ColMultiply = np.zeros(n-1)
    while not EncontrarMenorEnFila(Tabla,ColMultiply,NumeroVariables)[1]:
        indCol= EncontrarMenorEnFila(Tabla,ColMultiply,NumeroVariables)[0]
        pivote,indFila = EncontrarPivote(Tabla,indCol)
        FilaPorConstante(Tabla,indFila,1/pivote)
        ColumnaDelPivoteCero(Tabla,indFila,indCol)
        Solucion[indFila][0] = indCol+1
        ColMultiply[indFila] = Tabla[n-1][indCol]
    for i in range(Solucion.__len__()):
        if Solucion[i][0]!=0:
            Solucion[i][1] = Tabla[i][m-1]
    print(Solucion)

Y conseguimos el valor minimo con la siguiente función

In [31]:
def ConseguirValorMinimo(Solucion,z):
    VariablesColumna = np.zeros((z.__len__()))
    for i in range(z.__len__()):
        VariablesColumna[i] = Solucion[i][0]
    MinimoValor = 0
    for i in range(z.__len__()):
        indice = int(Solucion[i][0])-1
        if indice >= 0:
            MinimoValor += Solucion[i][1] * z[indice]
            print(f"El valor de x{indice+1} es: ",round(Solucion[i][1],1))
    if ArrayConCeros(VariablesColumna):
        print("Las otras variables son iguales a 0")
    return MinimoValor

Para ello se ejecutan varios ejemplos, recordando que deben de estar en la forma canónica

In [ ]:
# Funcion Main
import numpy as np

if __name__ == "__main__":
    # Ejemplo para maximizar
    A = [[-1,3],[7,1]]
    B = [[6],[35]]
    z = [7,9] # Maximar esa funcion 
    print("Ejemplo para maximizar")
    Solucion = np.zeros((z.__len__(),2))
    Tabla = ConstruirTablaMaximizar(A,B,z)
    print(Tabla)
    SolMaxima = SimplexMaximizar(Tabla,Solucion)
    print(Solucion)
    # Confirmar la Solucion optima
    ProbarOptimoValor(Solucion,z,SolMaxima)  

    # Tarea 3 de la PC1
    A = [[2,1,1],[1,2,3],[2,2,1]]
    B = [[2],[5],[6]]
    z = [3,1,3] # Maximar esa funcion 
    print("\nTarea 3 PC1 para maximizar")
    Solucion = np.zeros((z.__len__(),2))
    Tabla = ConstruirTablaMaximizar(A,B,z)
    print(Tabla)
    SolMaxima = SimplexMaximizar(Tabla,Solucion)
    print(Solucion)
    # Confirmar la Solucion optima
    ProbarOptimoValor(Solucion,z,SolMaxima) 

    # Ejemplo para minimizar
    A = [[1,4],[1,2]]
    B = [[3.5],[2.5]]
    z = [3,8]
    NumeroVariables = z.__len__()
    Tabla = ConstruirTablaMinimizar(A,B,z)
    Solucion = np.zeros((z.__len__(),2))
    print("\nEjemplo para minimizar")
    print(Tabla)
    SimplexMinimizar(Tabla,Solucion,NumeroVariables)
    MinimoValor = ConseguirValorMinimo(Solucion,z) 
    print("El Minimo Valor es: ",MinimoValor)

    # La tarea 4 de la PC1
    A = [[1,2,3],[2,2,1]]
    B = [[5],[6]]
    z = [3,4,5]
    NumeroVariables = z.__len__()
    Tabla = ConstruirTablaMinimizar(A,B,z)
    Solucion = np.zeros((z.__len__(),2))
    print("\nTarea 4 de la PC1")
    print(Tabla)
    SimplexMinimizar(Tabla,Solucion,NumeroVariables)
    MinimoValor = ConseguirValorMinimo(Solucion,z) 
    print("El Minimo Valor es: ",MinimoValor)


Ejemplo para maximizar
[[ 0. -1.  3.  1.  0.  6.]
 [ 0.  7.  1.  0.  1. 35.]
 [ 1. -7. -9.  0.  0.  0.]]
[[2.  3.5]
 [1.  4.5]]
La variable x2 es igual a 3.5
La variable x1 es igual a 4.5
La solucion maxima es :  63.0

Tarea 3 PC1 para maximizar
[[ 0.  2.  1.  1.  1.  0.  0.  2.]
 [ 0.  1.  2.  3.  0.  1.  0.  5.]
 [ 0.  2.  2.  1.  0.  0.  1.  6.]
 [ 1. -3. -1. -3.  0.  0.  0.  0.]]
[[1.  0.2]
 [3.  1.6]
 [0.  0. ]]
La variable x1 es igual a 0.2
La variable x3 es igual a 1.6
Las otras variables son iguales a 0
La solucion maxima es :  5.4

Ejemplo para minimizar
[[ 1.   4.  -1.   1.   0.   0.   3.5]
 [ 1.   2.   0.   0.  -1.   1.   2.5]
 [ 3.   8.   0.   1.   0.   1.   0. ]]


Por cada ejemplo se ve la tabla resultante, la matriz Solucion, en la que en la primera columna son los indices de las variables y en la segunda los valores de cada uno de ellos. Y el valor de cada variable y el valor máximo o mínimo dependiendo de cada caso